# Analog Output with AOTask

This notebook demonstrates analog output signal generation using the `AOTask` class.

In [ ]:
# Verify nidaqmx is installed
try:
    import nidaqmx
except ImportError:
    raise RuntimeError(
        "nidaqmx is not installed. Install with: pip install nidaqmx"
    )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from nidaqwrapper import AOTask, list_devices

# Display connected hardware
devices = list_devices()
print("Connected NI-DAQmx devices:")
for i, dev in enumerate(devices):
    print(f"  [{i}] {dev['name']} ({dev['product_type']})")

if not devices:
    print("\nNo devices found. Connect hardware before proceeding.")

## Single Value Output

Set a DC voltage level on a single analog output channel.

In [ ]:
# Edit these to match your hardware
device_ind = 0  # Index into list_devices() output
channel_ind = 0  # AO channel number (e.g., 0 for ao0)

task = AOTask("dc_output", sample_rate=1000)
task.add_channel("ao_0", device_ind=device_ind, channel_ind=channel_ind)
task.start(start_task=True)

# Allow buffer to fill
time.sleep(0.5)

# Generate single 2.5V DC value (shape: (1, 1))
dc_signal = np.array([[2.5]])
task.generate(dc_signal)

print(f"Output: {dc_signal[0, 0]} V DC on ao{channel_ind}")

task.clear_task()

## Sine Wave Generation

Generate a continuous sine wave on a single AO channel.

In [ ]:
device_ind = 0
channel_ind = 0

sample_rate = 10000  # Hz
freq = 100  # Hz
duration = 1.0  # seconds
amplitude = 5.0  # V

# Generate sine wave: shape (n_samples, 1)
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
signal = amplitude * np.sin(2 * np.pi * freq * t)
signal = signal.reshape(-1, 1)  # (n_samples, n_channels)

task = AOTask("sine_gen", sample_rate=sample_rate)
task.add_channel("sine_ao", device_ind=device_ind, channel_ind=channel_ind)
task.start(start_task=True)

time.sleep(0.5)

task.generate(signal)

print(f"Generated {freq} Hz sine wave, {amplitude} V amplitude")
print(f"Data shape: {signal.shape} (n_samples, n_channels)")

# Plot the waveform
plt.figure(figsize=(10, 4))
plt.plot(t[:500], signal[:500, 0])  # Plot first 500 samples
plt.xlabel("Time (s)")
plt.ylabel("Voltage (V)")
plt.title(f"{freq} Hz Sine Wave")
plt.grid(True)
plt.show()

task.clear_task()

## Multi-Channel Output

Generate different waveforms on two AO channels simultaneously.

In [ ]:
device_ind = 0
channel_0 = 0
channel_1 = 1

sample_rate = 10000
duration = 1.0

# Channel 0: 50 Hz sine
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
sig_0 = 3.0 * np.sin(2 * np.pi * 50 * t)

# Channel 1: 150 Hz sine
sig_1 = 2.0 * np.sin(2 * np.pi * 150 * t)

# Combine into (n_samples, 2) array
signal = np.column_stack((sig_0, sig_1))

task = AOTask("multi_ao", sample_rate=sample_rate)
task.add_channel("ao_0", device_ind=device_ind, channel_ind=channel_0)
task.add_channel("ao_1", device_ind=device_ind, channel_ind=channel_1)
task.start(start_task=True)

time.sleep(0.5)

task.generate(signal)

print(f"Generated multi-channel output:")
print(f"  Channel 0: 50 Hz, 3.0 V amplitude")
print(f"  Channel 1: 150 Hz, 2.0 V amplitude")
print(f"Data shape: {signal.shape}")

# Plot both channels
fig, axes = plt.subplots(2, 1, figsize=(10, 6))
axes[0].plot(t[:500], signal[:500, 0])
axes[0].set_ylabel("CH0 (V)")
axes[0].set_title("50 Hz Sine")
axes[0].grid(True)

axes[1].plot(t[:500], signal[:500, 1])
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("CH1 (V)")
axes[1].set_title("150 Hz Sine")
axes[1].grid(True)

plt.tight_layout()
plt.show()

task.clear_task()

## Context Manager

Use the `with` statement for automatic cleanup.

In [ ]:
device_ind = 0
channel_ind = 0

sample_rate = 5000
t = np.linspace(0, 0.5, int(sample_rate * 0.5), endpoint=False)
signal = 4.0 * np.sin(2 * np.pi * 200 * t).reshape(-1, 1)

with AOTask("ctx_ao", sample_rate=sample_rate) as task:
    task.add_channel("ao_ch", device_ind=device_ind, channel_ind=channel_ind)
    task.start(start_task=True)
    time.sleep(0.5)
    task.generate(signal)
    print("Signal output complete. Task will auto-cleanup.")

print("Exited context manager. Task is cleared.")

## Save/Load Config

Save task configuration to TOML and reload it.

In [ ]:
device_ind = 0
channel_ind = 0

# Create and configure task
task = AOTask("saved_ao", sample_rate=8000)
task.add_channel("output_ch", device_ind=device_ind, channel_ind=channel_ind, min_val=-5.0, max_val=5.0)

# Save to TOML (relative path, appears next to notebook)
config_path = "ao_config.toml"
task.save_config(config_path)
print(f"Saved config to {config_path}")

task.clear_task()

# Display config contents
with open(config_path, "r") as f:
    print("\nConfig file contents:")
    print(f.read())

# Reload from config
restored_task = AOTask.from_config(config_path)
print(f"\nRestored task: {restored_task.task_name}")
print(f"Channels: {restored_task.channel_list}")
print(f"Sample rate: {restored_task.sample_rate} Hz")

restored_task.clear_task()